In [9]:
search_for = 77
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003106355667114258
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 45110105
type: [1, 1, 1, 1, 77] 77
cases of this type: 35153041
10000 7.846480916262402
20000 13.983103262209497
30000 26.44832872813039
40000 23.783038334865832
50000 24.59607168877572
60000 23.803233904405
70000 23.929176981034512
80000 21.89007455155969
90000 23.036478288457623
100000 22.649626519214213
110000 23.060307069777448
120000 26.831710501833825
130000 23.13274141899652
140000 22.5863517979
150000 23.025429688779447
160000 24.16962579576135
170000 21.806219684019684
180000 24.11693991227577
190000 21.9435450281977
200000 21.16758301035146
210000 24.561373112951717
220000 11.349616176376177
230000 4.0760272417371475
240000 4.10055173036125
250000 19.92099272596869
260000 22.650043902256787
270000 21.56831944545504
280000 22.54303367194841
290000 22.444745313370525
300000 21.585475538811885
310000 21.653756263025073
32

3060000 17.96322975928502
3070000 18.14938451016903
3080000 17.190544509127534
3090000 17.094170761015416
3100000 17.052486506102817
3110000 17.12122572150588
3120000 16.983552561847336
3130000 17.381184776008094
3140000 18.414713592681586
3150000 11.329211432008412
3160000 8.440972414283058
3170000 8.586994075714944
3180000 18.809879848281444
3190000 17.518906304105386
3200000 19.5914454776449
3210000 19.32854794315345
3220000 17.383627387647493
3230000 15.468469877960084
3240000 6.4482567228006324
3250000 6.947847964828809
3260000 14.14016473029504
3270000 17.571408332274853
3280000 17.38969377233704
3290000 18.76346403213928
3300000 16.924897935208946
3310000 16.35222984075036
3320000 17.668774382130504
3330000 17.786566378817653
3340000 18.44863743077861
3350000 16.638905781231323
3360000 16.723966827809978
3370000 16.547861434771416
3380000 16.47829418479532
3390000 15.796332869532737
3400000 15.67415057994806
3410000 17.399098335864842
3420000 3.569832286062704
3430000 3.62242466

6140000 12.94680770567119
6150000 12.570963816922307
6160000 3.1518224099309244
6170000 3.311722862246434
6180000 5.915889737235407
6190000 16.342605449049472
6200000 13.341150104662479
6210000 14.38403013932642
6220000 13.717572394623526
6230000 14.349402079164982
6240000 15.129914224348365
6250000 14.655204534744817
6260000 13.637170741548537
6270000 13.830773470532531
6280000 14.910774593498774
6290000 14.633087371457817
6300000 13.017458432404895
6310000 7.034664003709986
6320000 7.169676966820559
6330000 10.908542852926486
6340000 16.917635008296404
6350000 14.679089582948885
6360000 15.601279243552089
6370000 14.651543676157395
6380000 13.544905262338519
6390000 16.141311124149293
6400000 16.42438177434547
6410000 16.19887064479351
6420000 13.955376280200216
6430000 15.7478983798131
6440000 15.46217531815072
6450000 14.12312241129021
6460000 14.949226234677303
6470000 9.179045498804747
6480000 6.338056047683226
6490000 6.495101972774764
6500000 14.780612114889125
6510000 14.54619

9220000 13.969757508743537
9230000 12.173729565998016
9240000 13.851739056927428
9250000 5.895448809436592
9260000 5.066384445790649
9270000 7.987792871883213
9280000 12.827753987080488
9290000 12.936285257187485
9300000 14.16344594595025
9310000 14.562011489248873
9320000 14.057842683853707
9330000 11.845469814992713
9340000 14.821100177312228
9350000 8.545323308197856
9360000 2.9012195841066046
9370000 2.7512141584741077
9380000 12.024134037375747
9390000 11.96642247763137
9400000 12.91872859080467
9410000 13.97950902064979
9420000 12.937873617533777
9430000 9.874762601950202
9440000 13.905362544163761
9450000 13.292034346616367
9460000 7.569172328347001
9470000 7.224293386939565
9480000 6.882559702594313
9490000 14.500682256073057
9500000 13.481090329708953
9510000 13.472361659354766
9520000 13.202394233772258
9530000 14.794628575085401
9540000 13.440993246838087
9550000 14.323132349431978
9560000 14.468176480462155
9570000 14.33673774302082
9580000 17.038818249236876
9590000 16.849

12220000 11.039155989527735
12230000 10.89372275331229
12240000 11.39686779076169
12250000 12.850387392468255
12260000 11.431091076105634
12270000 12.908040175129706
12280000 12.403536796930632
12290000 10.250909046767056
12300000 12.241889192359778
12310000 13.02307608205464
12320000 11.356046582232118
12330000 12.475547277131113
12340000 11.79807708134211
12350000 11.053451849030157
12360000 11.453175458857418
12370000 13.177381638530557
12380000 12.346929086015027
12390000 10.731789730322163
12400000 12.085983722623356
12410000 9.531736448810399
12420000 11.706485196897123
12430000 10.36775032880591
12440000 9.650006567768752
12450000 11.579077980313633
12460000 9.254101433502866
12470000 13.23642880312592
12480000 7.614170564340618
12490000 4.035814161443313
12500000 5.45013289776355
12510000 7.0768527196435125
12520000 11.563224991030527
12530000 10.749678389762243
12540000 11.758678961444994
12550000 2.8820923106881646
12560000 2.8568605808184544
12570000 2.8158557278936773
12580

...

33280000 4.014932601597805
33290000 2.361116851690411
33300000 1.0289616165490283
33310000 1.0184792056053547
33320000 2.1948967181725303
33330000 3.807658861525258
33340000 2.1483812758307326
33350000 1.4710226586769026
33360000 1.5333739799814425
33370000 3.424491115126047
33380000 4.079017792350948
33390000 3.7384585830292103
33400000 4.297246327928504
33410000 2.319751105882625
33420000 2.542573115583956
33430000 3.969548683627181
33440000 3.9910300019066534
33450000 2.443208241047164
33460000 3.8593179308253855
33470000 3.1204679709711667
33480000 3.778832673532566
33490000 2.210905817611284
33500000 2.7184226487742866
33510000 3.191538813661999
33520000 4.3511485549009175
33530000 3.473816823739608
33540000 4.408597220800552
33550000 0.55758672608157
33560000 0.5680968619641662
33570000 0.821714782447815
33580000 3.5072657433829373
33590000 2.7797219147556023
33600000 2.4463080616823167
33610000 3.8588464885828557
33620000 2.779014057223797
33630000 2.7510491484092343
33640000 2.

36230000 1.4295547018822032
36240000 1.1437716645206015
36250000 0.5549157494875788
36260000 0.4828826447811723
36270000 0.665335078364975
36280000 0.6473162639637456
36290000 0.6655051636064052
36300000 0.5065771702595221
36310000 0.5825154719956054
36320000 0.631251790797611
36330000 0.6133702394628193
36340000 0.46046910457740226
36350000 0.5718967877954244
36360000 0.806114652841058
36370000 1.0049844096260931
36380000 0.948689087724785
36390000 0.9975367799908916
36400000 0.7780300499353806
36410000 1.8296052472191056
36420000 0.8148905467918516
36430000 0.871624223440786
36440000 1.0742842583870889
36450000 1.205453740635945
36460000 1.3255169858060611
36470000 0.9616783472300569
36480000 0.9327477183738684
36490000 1.388063833494352
36500000 0.7831784520453215
36510000 0.6975595909048783
36520000 0.8687138661814399
36530000 0.8072124523678422
36540000 0.8251865118965176
36550000 1.0438220115447376
36560000 1.351579151780804
36570000 1.0286780442952117
36580000 1.329029323383901


39170000 0.6530081663856903
39180000 0.8073809042125278
39190000 1.1831983209360306
39200000 0.494322351885736
39210000 0.5531407831274139
39220000 0.5531314052522514
39230000 1.2144364359179138
39240000 1.060621204735173
39250000 0.9081463526552916
39260000 1.1793489057060083
39270000 1.0776529156260026
39280000 0.5702003496938944
39290000 0.7660502941556772
39300000 1.2409809391878048
39310000 1.0852468536973994
39320000 0.9555397393935918
39330000 0.709718529872497
39340000 0.8657649963115321
39350000 0.8758200652155279
39360000 0.6412763043126133
39370000 0.6740566041699051
39380000 0.7012992326780161
39390000 0.7235511032305493
39400000 0.7785834577773014
39410000 0.47812443356931206
39420000 0.7011929223259621
39430000 0.51253427691006
39440000 0.7794691484063865
39450000 0.36525753126925886
39460000 0.33455724937316444
39470000 0.5299106504614155
39480000 0.39401741146756547
39490000 0.4428776515186164
39500000 0.4917647582596541
39510000 0.5913531712771456
39520000 0.7670209627

42070000 0.532998887999687
42080000 0.34691617684443793
42090000 0.3397408270677593
42100000 0.21215449985593554
42110000 0.28278472300976515
42120000 0.4433956950389014
42130000 0.5461397177656824
42140000 0.35568275034656127
42150000 0.3917113427676758
42160000 0.33285099770959875
42170000 0.3232636861950159
42180000 0.3456143523616923
42190000 0.4218280100846952
42200000 0.4751734993237257
42210000 0.4543591454162201
42220000 0.2591110487462084
42230000 0.2741608816200495
42240000 0.3248866735896468
42250000 0.33490030195044146
42260000 0.4583661571740111
42270000 0.49409802482969234
42280000 0.4002641318302022
42290000 0.17486368281066417
42300000 0.2251966816729638
42310000 0.30700225748545595
42320000 0.3227267671946685
42330000 0.27564934956808884
42340000 0.44483255838500124
42350000 0.3792120441629489
42360000 0.33501298138631713
42370000 0.3083561006740398
42380000 0.33776930530250077
42390000 0.4355474382124014
42400000 0.6131491935456461
42410000 0.566647880881826
42420000 

44840000 0.020745266635020573
44850000 0.019317837301923168
44860000 0.023545591927601233
44870000 0.05260390433609485
44880000 0.04398319229016701
44890000 0.04484826566722658
44900000 0.02764843260973692
best so far: 0
type: [1, 1, 11, 7, 1] 77
cases of this type: 41503
44910000 0.013514493329226971
44920000 0.00957542350437906
44930000 0.00814448006093502
44940000 0.006352539474368096
best so far: 0
type: [1, 1, 77, 1, 1] 77
cases of this type: 5929
best so far: 0
type: [1, 7, 1, 1, 11] 77
cases of this type: 102487
44950000 0.0073798618986871505
44960000 0.00983819123506546
44970000 0.016692236236532532
44980000 0.013061158724195426
44990000 0.009204931110739707
45000000 0.008437052605549493
45010000 0.0077641306356589
45020000 0.007066400792797407
45030000 0.008477099450329938
45040000 0.007703914409147367
45050000 0.007741778842906156
best so far: 0
type: [1, 7, 1, 11, 1] 77
cases of this type: 9317
best so far: 0
type: [1, 7, 11, 1, 1] 77
cases of this type: 847
45060000 0.00343